In [ ]:
#%load_ext autoreload
#%autoreload 2

# <center> Titan2D Hazard Map Emulator Workflow</center>

<img src="data/p04pzpgn.jpg" width="600" height="600" align="center"></img>
#### <center><figcaption><br>Colima Volcano, Mexico<br>October, 2016<br>BBC News</figcaption></center>

In [ ]:
# Image from https://www.bbc.com/news/av/world-latin-america-38674298

## Overview

This workflow tool produces [Titan2D](https://github.com/TITAN2D/titan2d) hazard maps that display the probability of a volcanic flow depth reaching a critical height following a premonitory event.

Titan2D is a computer model for simulating granular avalanches over digital elevation models (DEMs) of natural terrain. The Titan2D hazard maps are constructed by creating a statistical surrogate model of the Titan2D computer model, requiring numerous executions of the Titan2D computer model and the emulator's uncertainy quantification (UQ) software. See [Workflows for Construction of Spatio-Temporal Probabilistic Maps for Volcanic Hazard Assessment](https://www.frontiersin.org/journals/earth-science/articles/10.3389/feart.2021.744655) for more information.

The [Pegasus Workflow Management System (WMS)](https://pegasus.isi.edu)  provides the structured platform for automating and managing these numerous executions, including staging the jobs, distributing the work, submitting the jobs to run in parallel, as well as handling data flow dependencies and overcoming job failures. This tool is designed to follow the Pegasus WMS, Amazon AWS Batch Deployment Scenario, which, in turn, is based on the AWS Fetch & Run Procedure. See [Welcome to Pegasus WMS’s documentation!](https://pegasus.isi.edu/documentation) and [Creating a Simple "Fetch & Run" AWS Batch Job](https://aws.amazon.com/blogs/compute/creating-a-simple-fetch-and-run-aws-batch-job/) for more information. 

This tool requires that you complete the prerequisites for configuring AWS Batch, which include creating an IAM user account with administrative access, creating required IAM roles and key pairs, and creating a Virtual Private Cloud (VPC) and security group.  See [Complete the AWS Batch prerequisites](https://docs.aws.amazon.com/batch/latest/userguide/get-set-up-for-aws-batch.html) for information on how to do this.

Specific IAM roles required for the AWS Fetch & Run Procedure are an AWS BATCH Service Role named AWSBatchServiceRole, an Elastic Container Service (ECS) Instance Role named ecsInstanceRole, and an IAM Role named batchJobRole. See the Pegasus WMS [Deployment Scenarios](https://pegasus.isi.edu/documentation/user-guide/deployment-scenarios.html), AWS Batch documentation, for more information.

AWS Batch comprises four components for a workflow: a compute environment, a job definition, a job queue, and the jobs. The Pegasus WMS provides an interface for creating and managing these AWS Batch components. To do this, Pegasus requires an AWS credential file, an AWS S3 configuration file, and JSON-formatted information catalogs. These files contain fields that reference your defined AWS Batch configurations. See the Pegasus WMS [Deployment Scenarios](https://pegasus.isi.edu/documentation/user-guide/deployment-scenarios.html), AWS Batch documentation, for more information. When this tool was configured, these files were configured with your AWS authorization credentials.

Follow the steps in the `Processing Steps` section to set up and run a workflow. Results from running the workflow are copied to the mounted shared storage directory, ./submithost/emulator/LOCAL/shared-storage.


In [ ]:
# Setup and preprocessing:

import atexit
import boto3
import datetime
import getpass
import math
import numpy as np
import os
import pandas as pd
import platform
import shutil
import subprocess
import sys
import time

import ipywidgets as widgets
from IPython.display import display, HTML, Markdown, clear_output, Image, Javascript
#import xml.etree.ElementTree as et

#from sklearn.neighbors import DistanceMetric

import hublib
#print (help(hublib))
import hublib.ui as ui
#print (help(ui))
#import hublib.use
#print (help(hublib.use))

import warnings
# The default Python for rockylinux:8 is Python 3.6.
# boto3 is returning a deprecation warning for Python 3.6
warnings.filterwarnings("ignore") 

# Set up the environment for this notebook

# Setup paths to executables
scriptpath = os.path.realpath(" ")
        
# Get the parent dirs
self_tooldir = os.path.dirname(scriptpath)

# Setup path to Bash, MATLAB, and Python scripts.
self_bindir = os.path.join(self_tooldir, "bin")

# Add to PYTHONPATH
sys.path.insert (1, self_bindir)

# Setup path to python and bash scripts and puffin executables
#self_libdir = os.path.join(self_tooldir, "lib")

# Add to PYTHONPATH
#sys.path.insert (2, self_libdir)

# Setup path to Pegasus WMS scripts
self_scriptsdir = os.path.join(self_tooldir, "pegasus-wms-scripts")

import hublib
#print (help(hublib))
import hublib.ui as ui
#print (help(ui))
#import hublib.use
#print (help(hublib.use))

# Setup paths to get data specific for this workflow
self_datadir = os.path.join(self_tooldir, "data")

self_examplesdir = os.path.join(self_tooldir, "examples")

# Set up path to the current session directory
self_workingdir = os.getcwd()

# Set up path to the user's home directory
#self_homedir = os.path.expanduser("~")

# Initialize the dated run directory.
# Workflow results are not available until after a workflow is executed via Pegasus and completes
#self_rundir = ""

#self_user = getpass.getuser()

#self_geo_location = GeoLocation.GeoLocation(0.0, 0.0, 0.0, 0.0)

#from Utils import Coordinates
from Utils.deg2utm import deg2utm
from Utils import GeoLocation
from Utils import Tree

#from newthreading import Thread
#from Wrapper import Wrapper
import Wrapper
from view_phm import view_phm

np.set_printoptions(threshold=np.inf)    

# Set up path to the current session directory
self_workflow_results_directory = os.path.join(self_workingdir, 'LOCAL', 'shared-storage')

self_log_filepath = os.path.join(self_workingdir, 'emulator_log_file.txt')
self_log_final_filepath = os.path.join(self_workflow_results_directory, 'emulator_log_file.txt')
self_log_backup_filepath = os.path.join(self_workingdir, 'emulator_log_backup_file.txt')

self_pegasus_analysis_file = os.path.join(self_workflow_results_directory, 'pegasus-analysis.txt')
self_pegasus_statistics_file = os.path.join(self_workflow_results_directory, 'pegasus-statistics.txt')
self_elevation_grid_file = os.path.join(self_workflow_results_directory,'elevation.grid')
self_final_phm_file = os.path.join(self_workflow_results_directory,'AZ_vol_dir_bed_int_final.phm')

BOLD = '\033[1m'
SUCCESS = '\033[92m'
WARNING = '\033[93m'
FAIL = '\033[91m'
END = '\033[0m'

dropdown_str_width = 20

dropdown_width = '850px'
dropdown_height = '25px'
button_width = '250px'
button_height = '40px'
ui_string_width = '94.8%'
ui_number_width = '98.7%'
ui_integer_width = '98.7%'
ui_dropdown_width = '96.2%'

# Clean up: remove files from the data/results folder and the bin/__pycache__ folder
def exit_handler():
    
    for file in os.listdir(self_workingdir):
        
        if os.path.isfile(file):
            if file.endswith(".txt"):
                if file != "README.txt":
                    print ("Deleting: %s\n" %file)
                    #os.remove(file)
            elif file.endswith(".png"):
                print ("Deleting: %s\n" %file)
                os.remove(file)
            elif file.endswith(".dax"):
                print ("Deleting: %s\n" %file)
                os.remove(file)
            elif file.endswith(".stdout"):
                print ("Deleting: %s\n" %file)
                #os.remove(file)
            elif file.endswith(".stderr"):
                print ("Deleting: %s\n" %file)
                #os.remove(file)

    dirpath = os.path.join(self_bindir, "__pycache__")
    if (os.path.exists(dirpath)):
        print ("Deleting: %s\n" %dirpath)
        shutil.rmtree(dirpath)
        
    FH1.close()
    shutil.move(self_log_filepath, self_log_final_filepath)
    

atexit.register(exit_handler);   

In [ ]:
# prevent In[] and Out[] from displaying on left
#HTML('''
#<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>
#''')

In [ ]:
#https://api.jquery.com/ready/
HTML('''
<script>
    function scroll_to_top() {
        Jupyter.notebook.scroll_to_top();
    } 
    $( window ).on( "load", scroll_to_top() );
</script>
''')

In [ ]:
# Button styles
HTML('''
<style>.buttontextclass { color:black ; font-size:110%}</style>
''')

In [ ]:
#https://stackoverflow.com/questions/36757301/disable-ipython-notebook-autoscrolling

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Initialize

if os.path.exists(self_log_filepath):
    shutil.copy (self_log_filepath, self_log_backup_filepath)
    
FH1 = open(self_log_filepath, 'w')

show_log_output_button = widgets.Button(description="Show Log Output", disabled=False,\
    layout=widgets.Layout(width=button_width, height=button_height),\
    style= {'button_color':'skyblue','font_weight':'bold'})

# Create the boto3 clients
boto3_session = boto3.session.Session()
boto3_iam_client = boto3.client('iam')
boto3_ec2_client = boto3.client('ec2')

try:
    
    aws_region = boto3_session.region_name
    iam_user_dict = boto3_iam_client.get_user()
    #keys = [x for x in iam_user_dict]
    #print(keys)
    # ['User', 'ResponseMetadata']
    #what is the MiB unit
    #print ('iam_user_dict: ', iam_user_dict)
    aws_iam_username = iam_user_dict['User']['UserName']
    
    
except Exception as e:
        
    print ("boto3 Exception: %s\n" %str(e))
    print ("An IAM username is required to run this tool.")
    aws_iam_username = None

# Utility Function

def log_info (message):
    if show_log_output_button.description == 'Hide Log Output': 
        with log_output:
            print (message)    
    FH1.write('%s\n' %message)
    FH1.flush()
        
log_info ('widgets.__version__ %s' %widgets.__version__)

if (1):
    
    log_info ("Operating System Platform: " + platform.system() + " " + platform.release())
    log_info ("\n")
    
    log_info ("sys.path : " + str(sys.path))
    log_info (" ")
    
    log_info ("os.environ['PATH']: " + os.environ["PATH"])
    log_info (" ")
    
    log_info ('shutil.which("python3"): ' + shutil.which("python3"))
    log_info (" ")

    log_info ("Environment:\n")
    log_info ("scriptpath: " + scriptpath)
    log_info ("tooldir: " + self_tooldir)
    log_info ("bindir: " + self_bindir)
    log_info ("scriptsdir: " + self_scriptsdir)
    log_info ("datadir: " + self_datadir)
    log_info ("examplesdir: " + self_examplesdir)
    log_info ("workingdir: " + self_workingdir)
    #log_info ("homedir: " + self_homedir)
    log_info ("resultsdir: " + self_workflow_results_directory)
    #log_info ("user: " + self_user)
    log_info (" ")
    
    log_info ('self_workflow_results_directory: ' + self_workflow_results_directory)
    
    log_info ('aws_region: %s' %aws_region)
    log_info ('aws_iam_username: %s' %aws_iam_username)


<a name="top"></a>

## [**Processing Steps**](#Menu)<br />

1. [Select Volcano](#volcano_selection) <br />
2. [Set Volcano Eruption Parameters](#volcano_eruption_parameters) <br />
3. [Set Simulation Parameters](#volcano_simulation_parameters) <br />
4. [Run the Workflow and View the Workflow's Progress and Results](#run_the_workflow)<br />
5. [View Pegasus Analysis and Statistics](#view_pegasus_analysis_and_statistics)<br />
6. [View Log Output](#view_log_output)<br />


In [ ]:
# #219F hex = #8607 decimal
# This works also works for an up arrow: [$\tiny\uparrow$](#top)

<a name="volcano_selection"></a>
## Volcano Selection [&#8607;](#top)

Select a volcano using the `Volcano Name` dropdown. Enter the first character of the volcano name to expedite the search.

This tool downloads a Shuttle Radar Topography Mission (SRTM) 30 m Global 1 arc second V003 GeoTiff DEM for the selected volcano, translates the GeoTiff DEM to GRASS GIS raster format, and creates a GRASS GIS Database for Titan2D.


In [ ]:
volcanos = []

filepath = os.path.join(self_datadir, "volcanos.txt")

FH2 = open(filepath, "r")
for volcano in FH2:
    # Remove white space
    volcano.strip()
    values = volcano.split(":")
    if (values[0] != "Puff Volcano Listing File - do not remove this header\n"):
        values = [value.strip(" ") for value in values]
        volcanos.append(values)
FH2.close()

#print (len(volcanos))
#print (volcanos[1:5])

volcano_names = [row[0] for row in volcanos]
#print (type(volcanoNames))
#print (len(volcanoNames))
#print (volcanoNames)
#max_len_volcano_name = max(volcano_names, key = len)
#print (max_len_volcano_name) # CAMPI FLEGREI MAR SICILIA
#print (len(max_len_volcano_name)) # 25

# Default volcano
volcano_name = 'COLIMA VOLC COMPLEX'
#database_volcano_name = get_database_volcano_name(volcano_name)

#layout={'max-height': '100px', 'width': 'initial', 'justify_content': 'center', 'overflow-y': 'auto'})

volcano_dropdown = widgets.Dropdown(
    name='Volcano Name',
    description='Volcano Name',
    value=volcano_name,
    options=volcano_names,
    style = {'description_width': '90px'},
    layout={'height': '30px', 'width': '350px'})
    
#help(volcanoDropDown)

# Process information for the selected volcano

process_volcano_info_output = widgets.Output()

def process_volcano_info():
    
    global volcano_name
    global volcano_name_previous
    global volcano_lat
    global volcano_lat_hemisphere
    global volcano_lat_decimal_degrees
    global volcano_lat_utmn
    global volcano_lon
    global volcano_lon_hemisphere
    global volcano_lon_decimal_degrees
    global volcano_lon_utme
    global volcano_height
    global current_datetime
    global self_default_radiosonde_filepath

    with process_volcano_info_output:
        
        clear_output()
    
        volcano_name = volcano_dropdown.value
        #print (volcano_name)
        #print (type(volcano_name))
        volcano_index = volcano_names.index(volcano_name)
        #print (volcano_index)
        
        #print (volcanos[volcano_index])
        volcano_location = volcanos[volcano_index][1]
        volcano_lat = float(volcanos[volcano_index][2])
        volcano_lat_hemisphere = volcanos[volcano_index][3]
        volcano_lon = float(volcanos[volcano_index][4])
        volcano_lon_hemisphere = volcanos[volcano_index][5]

        log_info ("%s, %s" %(volcano_name, volcano_location))
        
        # Decimal degrees
        sign = 1.0
        if (volcano_lat_hemisphere == 'S'):
            sign = -1.0
            #south = True
        #else:
            #south = False
        volcano_lat_decimal_degrees = sign * volcano_lat
        log_info ('\nLatitude: %s [degrees north -90 to 90]' %str(volcano_lat_decimal_degrees))
        lat.value = round(volcano_lat_decimal_degrees, 2)
        sign = 1.0
        if (volcano_lon_hemisphere == 'W'):
            sign = -1.0
        volcano_lon_decimal_degrees = sign * volcano_lon
        log_info ('Longitude: %s [degrees east -180 to 180]' %str(volcano_lon_decimal_degrees))
        lon.value = round(volcano_lon_decimal_degrees, 2)
        
        # Check @ https://www.latlong.net/lat-long-utm.html
        volcano_lon_utme, volcano_lat_utmn, utmzonen, utmzonel = deg2utm(volcano_lat_decimal_degrees, volcano_lon_decimal_degrees)
        log_info ('Latitude: %s [UTMN]' %str(volcano_lat_utmn))
        log_info ('Longitude: %s [UMTE]' %str(volcano_lon_utme))
        utmzone = '%02d%c' %(utmzonen,utmzonel)
        log_info('utmzone: %s' %str(utmzone))

        # Get Lat/Lon Bounding box.
        # At the equator, 1 degree = 111 km
        # Expects lat in -90 to 90, lon in -180 to 180
        self_geo_location = GeoLocation(0.0, 0.0, 0.0, 0.0)
        geo_loc = self_geo_location.from_degrees (volcano_lat_decimal_degrees, volcano_lon_decimal_degrees)
        #print ('geo_loc: ', geo_loc)
         
        #111./2. = 55.5 [km] * .621371 = 34.486 [mi]
        #111./4. = 27.75 [km] * .621371 = 17.24 [mi]
        #111./8. = 13.875 [km] * .621371 = 8.62 [mi]
        halfSideInKm = 27.75
        SW_loc, NE_loc = geo_loc.bounding_locations(halfSideInKm)
        log_info ('type(SW_loc): %s SW_loc: %s' %(str(type(SW_loc)), str(SW_loc)))
        log_info ('type(NE_loc): %s NE_loc: %s' %(str(type(NE_loc)), str(NE_loc)))
        
        # For the hysplit model, srm_2_nc.py gets an error when the lat/lon range values are not rounded. 
        # The benthysplitwf tool should also get modified for this.
        lat_south.value = round(SW_loc.deg_lat,2)
        lat_north.value = round(NE_loc.deg_lat,2)
        lon_west.value = round(SW_loc.deg_lon,2)
        lon_east.value = round(NE_loc.deg_lon,2)
        
        #print (float(gridLatRangeStart.value))
        #print (float(gridLatRangeStop.value))
        #print (utils.convert_lon_minus180_180_to_0_360(float(gridLonRangeStart.value)))
        #print (utils.convert_lon_minus180_180_to_0_360(float(gridLonRangeStop.value)))
       
        volcano_height = float(volcanos[volcano_index][6].rstrip())
        log_info ('Height: %s [m]\n' %str(volcano_height))
        
        # Update grass_gis_database_form values
        #database_volcano_name = get_database_volcano_name(volcano_name)
        #grassgis_location.value = database_volcano_name + '_location'
        #grassgis_mapset.value = database_volcano_name + '_mapset'
        #grassgis_map.value = database_volcano_name + '_map'
        
def volcano_name_change(change):
    
    if change['type'] == 'change' and change['name'] == 'value' and change['new'] != ' ':
        process_volcano_info()

volcano_dropdown.observe(volcano_name_change)

volcano_selection_form = ui.Form([volcano_dropdown], name = 'Volcano Selection')

lat = ui.Number(
    name = 'Latitude',
    description = 'Latitude [degrees north -90 to 90]',
    units = 'deg',
    value = '0.0',
    min = '-90.0',
    max = '90.0',
    disabled = True
)
lat_south = ui.Number(
    name = 'Latitude South',
    description = 'Latitude South [degrees north -90 to 90]',
    units = 'deg',
    value = '0.0',
    min = '-90.0',
    max = '90.0'
)
lat_north = ui.Number(
    name = 'Latitude North',
    description = 'Latitude North [degrees north -90 to 90]',
    units = 'deg',
    value = '0.0',
    min = '-90.0',
    max = '90.0'
)
lon = ui.Number(
    name = 'Longitude',
    description = 'Longitude [degrees east -180 to 180]',
    units = 'deg',
    value = '0.0',
    min = '-180.0',
    max = '180.0',
    disabled = True
)
lon_west = ui.Number(
    name = 'Longitude West',
    description = 'Longitude West [degrees east -180 to 180]',
    units = 'deg',
    value = '0.0',
    min = '-180.0',
    max = '180.0'
)
lon_east = ui.Number(
    name = 'Longitude East',
    description = 'Longitude East [degrees east -180 to 180]',
    units = 'deg',
    value = '0.0',
    min = '-180.0',
    max = '180.0'
)

volcano_coordinates_form = ui.Form([lat, lon], name = 'Volcano Coordinates')

volcano_BB_coordinates_form = ui.Form([lat_south, lat_north, lon_west, lon_east], name = 'Volcano Bounding Box Coordinates')

volcano_parameters_form = \
    ui.Form([volcano_selection_form, volcano_coordinates_form, volcano_BB_coordinates_form], \
    name = 'Volcano Parameters')


In [ ]:
display (volcano_parameters_form)

In [ ]:
# The following section is retained for a future enhancement to allow the user to input their own GRASS GIS Database.

#<a name="grass_gis_database_selection"></a>
### GRASS GIS Database [&#8607;](#top)

# A GRASS GIS Database is a set of directories and files with certain structure which GRASS GIS works efficiently with.
# Location is a directory with data related to one geographic location or a project.
# All data within one Location has the same cartographic projection.
# A Location contains Mapsets and each Mapset contains data related to a specific task, user or a smaller project.
# Within each Location, a mandatory PERMANENT Mapset exists which can contain commonly used data within a Location such as base maps.
# The PERMANENT Mapset also contains metadata related to a Location such as projection.
# When GRASS GIS is started by Titan2D, it connects to the created GRASS GIS Database, Location and Mapset.
# See https://grass.osgeo.org/grass82/manuals/grass_database.html for more information.


In [ ]:
HBox_layout = widgets.Layout(height='40px', width='98%', display='flex', flex_flow='row', justify_content='flex-start')
grassgis_database_label = widgets.Label(
    value = 'Database',
    layout = widgets.Layout(width='1000px',height='30px')
)
grassgis_database = widgets.Text(
    value = 'grassdata',
    layout = {'height': '30px', 'width': '500px'},
    style = {'description_width':'80px'},
    disabled = True
)
grassgis_database_box = widgets.HBox(children=[grassgis_database_label, grassgis_database], layout = HBox_layout)
grassgis_location_label = widgets.Label(
    value = 'Location',
    layout = widgets.Layout(width='1000px',height='30px')
)
grassgis_location = widgets.Text(
    #value=database_volcano_name + '_location',
    value = 'location',
    layout = {'height': '30px', 'width': '500px'},
    style = {'description_width':'80px'},
    disabled = True
)
grassgis_location_box = widgets.HBox(children=[grassgis_location_label, grassgis_location], layout = HBox_layout)
# Getting errors when Mapset is not PERMANENT?
grassgis_mapset_label = widgets.Label(
    value = 'Mapset',
    layout = widgets.Layout(width='1000px',height='30px')
)
grassgis_mapset = widgets.Text(
    value = 'PERMANENT',
    #value = database_volcano_name + '_mapset',
    layout = {'height': '30px', 'width': '500px'},
    style = {'description_width':'80px'},
    disabled = True
)
grassgis_mapset_box = widgets.HBox(children=[grassgis_mapset_label, grassgis_mapset], layout = HBox_layout)
grassgis_map_label = widgets.Label(
    value = 'Map',
    layout = widgets.Layout(width='1000px',height='30px')
)
grassgis_map = widgets.Text(
    #value=database_volcano_name + '_map',
    value = 'map',
    layout = {'height': '30px', 'width': '500px'},
    style = {'description_width':'80px'},
    disabled = True
)
grassgis_map_box = widgets.HBox(children=[grassgis_map_label, grassgis_map], layout = HBox_layout)
grassgis_database_form = ui.Form(
    [grassgis_database_box, grassgis_location_box, grassgis_mapset_box, grassgis_map_box], name = 'Grass GIS Database')


In [ ]:
# display(grassgis_database_form)

<a name="volcano_eruption_parameters"></a>
## Volcano Eruption Parameters [&#8607;](#top)
Set the volcano's material model and pile parameters for the volcano's eruption.

In [ ]:
material_models = ["Coulomb", "TwoPhase-Pitman-Le", "Voellmy-Salm", "Pouliquen-Forterre"]
for i in range (len(material_models)):
    material_models[i] = "{0:{1}}".format(material_models[i],dropdown_str_width)

material_model = ui.Dropdown(
        name='Material Model',
        description="Material Model",
        units = '',
        value=material_models[0],
        options=material_models,
        width=ui_dropdown_width
)
int_frict_angle = ui.Number(
    name = 'Internal Friction Angle',
    description = 'Internal Friction Angle',
    units = 'deg',
    value = '30.0',
    min = '-90.0',
    max = '90.0'
)
material_model_form = ui.Form([material_model,
             int_frict_angle,
             ], name = 'Material Model Parameters')

pile_types = ["Paraboloid", "Cylinder"]

for i in range (len(pile_types)):
    pile_types[i] = "{0:{1}}".format(pile_types[i],dropdown_str_width)
    
pile_type = ui.Dropdown(
        name='Pile Type',
        description="Pile Type",
        units = '',
        value=pile_types[0],
        options=pile_types,
        width=ui_dropdown_width
)
orientation_angle = ui.Number(
    name = 'Orientation Angle',
    description = 'Orientation Angle',
    units = 'deg',
    value = '-45.0',
    min = '-90.0',
    max = '90.0'
)
initial_speed = ui.Number(
    name = 'Initial Speed',
    description = 'Initial Speed',
    units = 'm/s',
    value = '2.0',
    min = '1.0',
    max = '3.0'
)
initial_direction = ui.Number(
    name = 'Initial Direction',
    description = 'Initial Direction',
    units = 'deg',
    value = '-5.0',
    min = '-90.0',
    max = '90.0'
)

pile_form = ui.Form([pile_type,
             orientation_angle,
             initial_speed,
             initial_direction,
             ], name = 'Pile Parameters')

erupt_parameters_form = ui.Form([material_model_form ,
             pile_form,
             ], name = 'Eruption Parameters')

# On debian 10, datetime returns time in UTC.
#now = datetime.datetime.now()
#print ('time now: ', now)
utcnow = datetime.datetime.utcnow()
#print ('time now [UTC]: ', utcnow)
  
current_datetime = utcnow
# print ('current_datetime: ', current_datetime)

#eruptDatetime.value = str(datetime.datetime(current_datetime.year, current_datetime.month, current_datetime.day, current_datetime.hour, 0, 0))


In [ ]:
display(erupt_parameters_form)

In [ ]:
minvol = ui.Number(
    name = 'minvol',
    description = 'minvol',
    units = 'm^3',
    value = '1000000000',
    min = '1000',
    max = '10000000000'
)
maxvol = ui.Number(
    name = 'maxvol',
    description = 'maxvol',
    units = 'm^3',
    value = '10000000000',
    min = '1000',
    max = '10000000000'
)
minbed = ui.Number(
    name = 'minbed',
    description = 'minbed',
    units = 'deg',
    value = '20',
    min = '1',
    max = '100'
)
maxbed = ui.Number(
    name = 'maxbed',
    description = 'maxbed',
    units = 'deg',
    value = '28',
    min = '1',
    max = '100'
)
radius = ui.Number(
    name = 'radius',
    description = 'radius',
    units = 'm',
    value = '2500.00',
    min = '1',
    max = '5000.0'
)

eruption_simulation_form = ui.Form([minvol,
             maxvol,
             minbed,
             maxbed,
             radius], name = 'Simulation Parameters')



<a name="volcano_simulation_parameters"></a>
## Volcano Simulation Parameters [&#8607;](#top)
Set the minimum volume, maximum volume, minimum bed friction angle, maximum bed friction angle, and radius parameters for the emulator's UQ software.

In [ ]:
display(eruption_simulation_form)

In [ ]:
# VPCs are assigned for regions.
def get_subnet_info(vpc):
    
    subnets_dict = boto3_ec2_client.describe_subnets(
        Filters=[{'Name': 'vpc-id', 'Values': [vpc]}]
    )
    #keys = [x for x in subnets_dict]
    #print(keys)
    # ['Subnets', 'ResponseMetadata']
    log_info ('subnets_dict: %s' %str(subnets_dict))
    subnets = [ sub['SubnetId'] for sub in subnets_dict['Subnets']]
    subnet.options = subnets
    subnet.value = subnets[0]

def get_security_group_info(vpc):
    
    security_groups_dict = boto3_ec2_client.describe_security_groups(
        Filters=[{'Name': 'vpc-id', 'Values': [vpc]}]
    )
    #keys = [x for x in security_groups_dict]
    #print(keys)
    # ['SecurityGroups', 'ResponseMetadata']
    #print (security_groups_dict)
    log_info ('security_groups_dict: %s' %str(security_groups_dict))
    security_groups = [ sub['GroupId'] for sub in security_groups_dict['SecurityGroups']]
    security_group.options = security_groups
    security_group.value = security_groups[0]
    
def get_vpc_info():

    vpcs_dict = boto3_ec2_client.describe_vpcs()
    #keys = [x for x in vpcs_dict]
    #print(keys)
    # ['Vpcs', 'ResponseMetadata']
    #print (vpcs_dict)
    log_info ('vpcs_dict: %s' %str(vpcs_dict))

    #keys = [x for x in vpcs_dict]
    #print(keys)
    # ['Vpcs', 'ResponseMetadata']

    vpcs = [ sub['VpcId'] for sub in vpcs_dict['Vpcs']]
    #print('vpcs: ', vpcs)
    
    vpc.options = vpcs
    vpc.value = vpcs[0]

    get_subnet_info(vpcs[0])
    get_security_group_info(vpcs[0])
    
def vpc_dropdown_callback(change):
    
    if change['type'] == 'change' and change['name'] == 'value' and change['new'] != ' ' \
        and vpc.value != None:
        
        runWorkflowButton.disabled = True
        
        with workflow_progress_output:
            
            clear_output()
        
            #print ('p: ', p)
            #print ('value: ', value)

            try:

                selected_vpc = vpc.value
                #print ('selected_vpc: ', selected_vpc)
                get_subnet_info(selected_vpc)
                get_security_group_info(selected_vpc)

            except Exception as e:
                print ("vpc_dropdown_callback: Exception: %s\n" %str(e))
                
        runWorkflowButton.disabled = False


In [ ]:
# Workflow Parameters
# ui.Dropdown widgets do not allow dynamic changing of the options
DROPDOWN_WIDTH = '200px'
DROPDOWN_HEIGHT = '44px'
DESCRIPTION_WIDTH = '700px'
ui_integer_width = '98.7%'
ui_dropdown_width = '99.5%'

eruptsimulations = ui.Integer(
    name = 'Number of Simulations',
    description = 'Number of Simulations',
    units = '',
    value = 32,
    min = 32,
    max = 1024,
    layout = widgets.Layout(width=ui_integer_width)
)
memory = ui.Integer(
    name = 'Memory',
    description = '',
    units = 'MiB',
    value = 600,
    min = 600,
    max = 1000,
    layout = widgets.Layout(width=ui_integer_width)
)

max_description_length = 189
vpc_description = 'VPC'
for i in range(max_description_length - len(vpc_description)):
    vpc_description = vpc_description + '\xa0'
vpc = widgets.Dropdown(
        description = vpc_description,
        units = '',
        options = [],
        value = None,
        style = {'description_width': DESCRIPTION_WIDTH},
        layout=widgets.Layout(width=ui_dropdown_width)
)
vpc.observe(vpc_dropdown_callback)

max_description_length = 187
subnet_description = 'Subnet'
for i in range(max_description_length - len(subnet_description)):
    subnet_description = subnet_description + '\xa0'
subnet = widgets.Dropdown(
        description = subnet_description,
        units = '',
        options = [],
        value = None,
        style = {'description_width': DESCRIPTION_WIDTH},
        layout=widgets.Layout(width=ui_dropdown_width)
)

max_description_length = 182
security_group_description = 'Security Group'
for i in range(max_description_length - len(security_group_description)):
    security_group_description = security_group_description + '\xa0'
security_group = widgets.Dropdown(
        description = security_group_description,
        units = '',
        options = [],
        value = None,
        style = {'description_width': DESCRIPTION_WIDTH},
        layout=widgets.Layout(width=ui_dropdown_width),
)
# The bid percentage is a percentage of the Amazon EC2 Spot Instance On-Demand price for compute resources
# that you a user is willing to pay for running each workflow job. Increasing the bid percentage should 
# speedup the time it takes to complete the workfow. Currently, the compute environments for the workflow 
# are not setup for Sport Instances.
'''
bid_percentage = ui.Integer(
    name = 'Bid Percentage',
    description = 'AWS Bid Percentage',
    units = '%',
    value = 20,
    min = 20,
    max = 100
)
'''
maxwalltime = ui.Integer(
    name = 'Maximum Walltime',
    description = 'Maximum Walltime [min]',
    units = 'min',
    value = '60',
    min = '10',
    max = '360'
)

run_parameters_form = ui.Form([eruptsimulations, maxwalltime], 
    name = 'Emulator Run Parameters')

compute_environment_parameters_form = ui.Form([vpc, subnet, security_group], 
    name = 'Compute Environment Parameters')

job_definition_form = ui.Form([memory], 
    name = 'Job Definition Parameter')



In [ ]:
def run_workflow(p):
    
    # print (p) #Button
    
    workflow_progress_output.clear_output()
    workflow_results_output.clear_output()
        
    with workflow_progress_output:
        
        print ('Workflow in progress...\n')
    
        runWorkflowButton.disabled = True
        resetVPCButton.disabled = True
        show_pegasus_analysis_button.disabled = True
        show_pegasus_statistics_button.disabled = True
        show_log_output_button.disabled = True
        
        start_time = time.time()

        try:
            
            if aws_iam_username !=None  and subnet.value != None and security_group.value != None:
                
                for file in os.listdir(self_workflow_results_directory):
                    #print ('removing: %s' %os.path.join(self_workflow_results_directory, file))
                    os.remove(os.path.join(self_workflow_results_directory, file))

                # Instantiate the Wrapper class 
                wrapper = Wrapper.Wrapper \
                    (self_workingdir, self_bindir, self_scriptsdir, self_datadir, self_examplesdir, self_workflow_results_directory, \
                    int(eruptsimulations.value), int(maxwalltime.value))            
                #print ('wrapper: ', rapper)

                # Initialize the workflow with user selected selected modeling parameters.
                #'''
                returnCode = wrapper.initialize_workflow(volcano_lat_decimal_degrees, volcano_lon_decimal_degrees, \
                    volcano_lat_utmn, volcano_lon_utme, \
                    lat_south.value, lat_north.value, lon_west.value, lon_east.value, \
                    grassgis_database.value, grassgis_location.value, grassgis_mapset.value, grassgis_map.value, \
                    material_model.value.rstrip(), int_frict_angle.value, \
                    pile_type.value.rstrip(), orientation_angle.value, initial_speed.value, initial_direction.value, \
                    minvol.value, maxvol.value, minbed.value, maxbed.value, radius.value,
                    memory.value, subnet.value, security_group.value)
                #'''
                #returnCode = False
                print ('wrapper.initialize_workflow returnCode: ', returnCode)

                if returnCode == True:

                    # Run the workflow
                    returnCode = wrapper.run_workflow(aws_region, aws_iam_username)
                    print ('wrapper.run_workflow returnCode: ', returnCode)

                    if returnCode == True:

                        if os.listdir(self_workflow_results_directory):

                            #print ('\nDirectory tree of ', self_workflow_results_directory, ':\n')
                            #results_tree = Tree(self_workflow_results_directory)
                            #results_tree.print_tree()
                            #print ('\n')

                            if os.path.exists(self_elevation_grid_file) and os.path.exists(self_final_phm_file):

                                with workflow_results_output:
                                    display_results()

                            else:

                                print ("Workflow did not complete successfully.\n")

                                if os.path.exists(self_elevation_grid_file) == False:
                                    print ("%s not generated by the workflow" %self_elevation_grid_file)

                                if os.path.exists(self_final_phm_file) == False:
                                    print ("%s not generated by the workflow" %self_final_phm_file)

                        else:

                            print ("Workflow did not complete successfully.")
                            print ("%s not generated by the workflow\n" %self_workflow_results_directory)

                            #print ("\nPlease see the log output\n")
                    else:
                            print ("Workflow did not complete successfully.")
                            print ("Worflow run failure\n")
                else:
                        print ("Workflow did not complete successfully.")
                        print ("Worflow initialization failure\n")

            else:
                
                print ("Workflow did not complete successfully\n")
                
                if aws_iam_username==None:
                    print ("An IAM username is required to run this tool.")
                if subnet.value==None:
                    print ("A VPC subnet is required to run this tool.")
                if security_group.value==None:
                    print ("A VPC security group is required to run this tool.")
                

        except Exception as e:
        
            print ("Workflow Exception: %s\n" %str(e))
            print ("Please see the log output\n")
       
        # Snapshot
        shutil.copy(self_log_filepath, self_log_final_filepath)
                
        runWorkflowButton.disabled = False
        resetVPCButton.disabled = False        
        show_pegasus_analysis_button.disabled = False
        show_pegasus_statistics_button.disabled = False
        show_log_output_button.disabled = False
            
        print ("\nWorkflow elapsed time: " + str((time.time() - start_time)/60.0) + " minutes\n")


# Abort
# Select Kernel Interrupt
#if self_tW.is_alive() == True:
   #self_tW.terminate()

runWorkflowButton = widgets.Button(description="Run Workflow", disabled=False,\
    layout=widgets.Layout(width=button_width, height=button_height),\
    style= {'button_color':'lawngreen','font_weight':'bold'})
runWorkflowButton.add_class("buttontextclass")
runWorkflowButton.on_click (run_workflow)
#help (runWorkflowButton)

In [ ]:
def reset_VPC(p):
    
    # print (p) #Button
    
    get_vpc_info()
    
resetVPCButton = widgets.Button(description="Reset VPC", disabled=False,\
    layout=widgets.Layout(width=button_width, height=button_height),\
    style= {'button_color':'darkseagreen','font_weight':'bold'})
resetVPCButton.add_class("buttontextclass")
resetVPCButton.on_click (reset_VPC)


In [ ]:

workflow_parameters_form = ui.Form([run_parameters_form, compute_environment_parameters_form, job_definition_form, \
    runWorkflowButton, resetVPCButton], 
    name = 'Run Workflow')



<a name="run_the_workflow"></a>
## Run the Workflow and View the Workflow's Progress and  Results [&#8607;](#top)

### Run the Workflow

Enter your workflow parameters and click the `Run Workflow` button to initiate the ensemble runs of Titan2D and the emulator's UQ software for your selected volcano and modeling parameters, and produce a PHM formatted file containing information on the probability of a volcanic flow depth reaching a critical height at specific locations.

- All VPCs that you have created for your Amazon AWS Region are available for selection using the `VPC` dropdown. Select a VPC from the `VPC` dropdown, and select a subnet and security group for the selected VPC using the `Subnet` and `Security Group` dropdowns. If you change your Amazon AWS Virtual Private Cloud (VPC) configuration, click the `Reset VPC` button to update the `VPC`,`Subnet` and `Security Group` dropdowns for your new configuration.  
    
- Set the amount of memory allocated for running each workflow job using the `Memory` textbox.
    
- This tool's directories contain templates for the files that Pegasus requires. When the `Run Worklow` button is clicked, a script is run that further configures the JSON-formatted information catalogs with your selected parameters.

### View the Workflow's Progress

- Messages generated while running the workflow are displayed in the `View Workflow Progress` section.

### View the Workflow's Results

- The PHM formatted file returned by the workflow is processed to create the Hazard_Report.pdf file. The Hazard_Report.pdf file is stored in the shared storage directory and also displayed in the `View Workflow Results` section.

- Standard output and error files returned from running the workflow are also stored in the shared storage directory.


In [ ]:
runworkflowoutput = widgets.Output(layout=widgets.Layout(border='1px solid black'))
display (runworkflowoutput)
with runworkflowoutput:
    display(workflow_parameters_form)


<a name="view_workflow_progress"></a>
### View Workflow Progress [&#8607;](#top)


In [ ]:
workflow_progress_output = widgets.Output(layout={'border': '1px solid black'})
display(workflow_progress_output)

<a name="view_workflow_results"></a>
### View Workflow Results [&#8607;](#top)


In [ ]:
def display_results():

    workflow_results_output.clear_output()
        
    with workflow_results_output:

        P_filename = "P.png";
        P_filepath = os.path.join(self_workflow_results_directory,P_filename)

        if (os.path.exists(P_filepath) == True):
            os.remove(P_filepath)

        SDP_filename = "SDP.png";
        SDP_filepath = os.path.join(self_workflow_results_directory,SDP_filename)

        if (os.path.exists(SDP_filepath) == True):
            os.remove(SDP_filepath)

        view_phm(self_workingdir, self_workflow_results_directory, volcano_name, volcano_lat_decimal_degrees, volcano_lon_decimal_degrees)
        
        if (os.path.exists(P_filepath) == True):
            display(Image(P_filepath))
        else:
            log_error (workflow_results, 'Image %s not created.' %P_filepath)
            
        if (os.path.exists(SDP_filepath) == True):
            display(Image(SDP_filepath))
        else:
            log_error (workflow_results, 'Image %s not created.' %SDP_filepath)
    



In [ ]:
workflow_results_output = widgets.Output(layout={'border': '1px solid black'})
display(workflow_results_output)

In [ ]:
# Uncomment for unit testing
#display_results()

<a name="view_pegasus_analysis_and_statistics"></a>
## View Pegasus Analysis and Statistics [&#8607;](#top)

- Pegasus maintains a log file containing the completion status of the workflow jobs. This log file is saved to the file pegasus-analysis.txt in the shared storage directory. If a workflow job fails, the cause of the failure is written to the pegasus-analysis.txt file. Click the `Show Pegasus Analysis` button to view the pegasus-analysis.txt file.

- Pegasus also maintains workflow statistics. These statistics are saved to the file pegasus-statistics.txt in the shared storage directory. Click the `Show Pegasus Satistics` button to view the pegasus-statistics.txt file.


In [ ]:
def show_pegasus_analysis():
    
    if show_pegasus_analysis_button.description == 'Hide Pegasus Analysis':

        pegasus_analysis_output.clear_output()        

        with pegasus_analysis_output:
            if os.path.exists(self_pegasus_analysis_file):
                #print("%s: \n" %self_var_criteria_tabulated_csv)
                f = open(self_pegasus_analysis_file,'r')
                for line in f:
                    print(line.rstrip())
                f.close()
            else:
                log_status (pegasus_analysis_output, '%s does not exist ' %self_pegasus_analysis_file)

def show_pegasus_analysis_button_on_click(change):
    
    if os.path.exists(self_pegasus_analysis_file):
            
        if show_pegasus_analysis_button.description == 'Show Pegasus Analysis':
            
            show_pegasus_analysis_button.description = 'Hide Pegasus Analysis'
            show_pegasus_analysis()
            
        else:
        
            show_pegasus_analysis_button.description = 'Show Pegasus Analysis'
            pegasus_analysis_output.clear_output()
    else:
        log_status (pegasus_analysis_output, '%s does not exist ' %self_pegasus_analysis_file)

show_pegasus_analysis_button = widgets.Button(description="Show Pegasus Analysis", disabled=False,\
    layout=widgets.Layout(width=button_width, height=button_height),\
    style= {'button_color':'skyblue','font_weight':'bold'})

show_pegasus_analysis_button.add_class("buttontextclass")
show_pegasus_analysis_button.on_click(show_pegasus_analysis_button_on_click)
display (show_pegasus_analysis_button)

In [ ]:
pegasus_analysis_output = widgets.Output(layout={'border': '1px solid black'})
display (pegasus_analysis_output)

In [ ]:
def show_pegasus_statistics():
    
    if show_pegasus_statistics_button.description == 'Hide Pegasus Statistics':

        pegasus_statistics_output.clear_output()        

        with pegasus_statistics_output:
            if os.path.exists(self_pegasus_statistics_file):
                #print("%s: \n" %self_var_criteria_tabulated_csv)
                f = open(self_pegasus_statistics_file,'r')
                for line in f:
                    print(line.rstrip())
                f.close()
            else:
                log_status (pegasus_statistics_output, '%s does not exist ' %self_pegasus_statistics_file)

def show_pegasus_statistics_button_on_click(change):
    
    if os.path.exists(self_pegasus_statistics_file):
            
        if show_pegasus_statistics_button.description == 'Show Pegasus Statistics':
            
            show_pegasus_statistics_button.description = 'Hide Pegasus Statistics'
            show_pegasus_statistics()
            
        else:
        
            show_pegasus_statistics_button.description = 'Show Pegasus Statistics'
            pegasus_statistics_output.clear_output()
    else:
        log_status (pegasus_statistics_output, '%s does not exist ' %self_pegasus_statistics_file)

show_pegasus_statistics_button = widgets.Button(description="Show Pegasus Statistics", disabled=False,\
    layout=widgets.Layout(width=button_width, height=button_height),\
    style= {'button_color':'skyblue','font_weight':'bold'})

show_pegasus_statistics_button.add_class("buttontextclass")
show_pegasus_statistics_button.on_click(show_pegasus_statistics_button_on_click)
display (show_pegasus_statistics_button)

In [ ]:
pegasus_statistics_output = widgets.Output(layout={'border': '1px solid black'})
display (pegasus_statistics_output)

<a name="view_log_output"></a>
## View Log Output [&#8607;](#top)

- This tool maintains a log output file, emulator_log_file.txt. Click the `Show Log Output` button to view the log output file.


In [ ]:
def show_log_output(change):
    
    if os.path.exists(self_log_filepath):
            
        if show_log_output_button.description == 'Show Log Output':
        
            show_log_output_button.description = 'Hide Log Output'
        
            with log_output:
            
                if os.path.exists(self_log_filepath):
                    print("%s: \n\n" %self_log_filepath)
                    f = open(self_log_filepath,'r')
                    for line in f:
                        print(line.rstrip())
                    f.close()
                else:
                    log_error (log_output, '%s does not exist ' %self_log_filepath + '. Please contact us.')
        else:
        
            show_log_output_button.description = 'Show Log Output'
            log_output.clear_output()
    else:
        log_error (log_output, '%s does not exist ' %self_log_filepath + '. Please contact us.')

show_log_output_button.add_class("buttontextclass")
show_log_output_button.on_click(show_log_output)
display (show_log_output_button)

In [ ]:
log_output = widgets.Output(layout={'border': '1px solid black'})
display (log_output)

In [ ]:
# Utility Functions

def log_status (output_widget, message):
    
    with output_widget:
        print (message)
    log_info (message)
    
def log_success (output_widget, message):
    
    with output_widget:
        print ('%s%s%s' %(SUCCESS,message,END))
    log_info (message)
    
def log_warning (output_widget, message):
    
    with output_widget:
        print ('%s%s%s' %(WARNING,message,END))
    log_info (message)
    
def log_error (output_widget, message):
    
    with output_widget:
        print ('%s%s%s' %(FAIL,message,END))
    log_info (message)

def disable_widgets():
    
    volcano_parameters_form.disabled = True
    #eruption_parameters_form.disabled = True
    #create_figures_button.disabled = True
    #show_log_output_button.disabled = True
    #downloadTXTButton.disabled = True
    
def enable_widgets():
    
    volcano_parameters_form.disabled = False
    lat.disabled = True
    lon.disabled = True
    #eruption_parameters_form.disabled = False
    #create_figures_button.disabled = True
    #show_log_output_button.disabled = False
    #downloadTXTButton.disabled = False
         
def initialize_forms():
    
    disable_widgets()
    
    # Process default volcano
    process_volcano_info()
    
    get_vpc_info()

    enable_widgets()
                        


In [ ]:
# Initialize forms and widgets with default values
initialize_forms()
